In [6]:
# First, install the library needed to split the folders
!pip install split-folders

import splitfolders
import os

# --- THIS IS THE CORRECTED CODE ---

# 1. Define the path to your original dataset
# This path is based on the output of your !ls command
input_folder = "/kaggle/input/imagesoasis/Data"

# 2. Define the path where the new split dataset will be created
output_folder = "/kaggle/working/split_data"

# 3. Split the data into a training set (75%) and a testing set (25%).75
splitfolders.ratio(input_folder, output=output_folder, seed=42, ratio=(.75, .25))

print("Dataset successfully split into train and test sets.")
print(f"Split data is now located in: {output_folder}")

Copying files: 86437 files [24:06, 59.77 files/s] 

Dataset successfully split into train and test sets.
Split data is now located in: /kaggle/working/split_data


In [7]:
!ls /kaggle/input/

imagesoasis


In [8]:
test_dir = os.path.join(output_folder, "test")

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# --- FIX 1: Use the correct path from your first cell ---
output_folder = "/kaggle/working/split_data"
# --------------------------------------------------------

# Define the directories
train_dir = os.path.join(output_folder, "train")

# --- FIX 2: Your 'splitfolders' created 'test', not 'val' ---
test_dir = os.path.join(output_folder, "test") 
# ----------------------------------------------------------

BATCH_SIZE = 32

# Create a generator for training and validation
# 15% of this data will be used for validation
train_val_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.15,  # Reserve 15% of the training data for validation
    
    # --- Added Augmentations ---
    rotation_range=15,      
    width_shift_range=0.1,  
    height_shift_range=0.1, 
    shear_range=0.1,        
    zoom_range=0.1,         
    horizontal_flip=True,   
    fill_mode='nearest'     
    # ---------------------------
)

# A separate generator for the test set (only rescaling)
test_datagen = ImageDataGenerator(rescale=1./255)

# Create the train generator from the 'training' subset
train_generator = train_val_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'  # Specify this is the training set
)

# Create the validation generator from the 'validation' subset
val_generator = train_val_datagen.flow_from_directory(
    train_dir, # Note: It uses the same directory
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation' # Specify this is the validation set
)

# Create the test generator from the corrected path ('test_dir')
test_generator = test_datagen.flow_from_directory(
    test_dir, # <-- This now correctly points to your 'test' folder
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

print("Data generators created successfully!")
# You can also check the number of images found
print(f"Found {train_generator.samples} images for training.")
print(f"Found {val_generator.samples} images for validation.")
print(f"Found {test_generator.samples} images for testing.")

Found 55105 images belonging to 4 classes.
Found 9721 images belonging to 4 classes.


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/split_data/test'

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
# Make sure to import the Dropout layer
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Concatenate, Flatten, Dense, Dropout

# Input Layer
inputs = Input(shape=(224, 224, 3))

# ======================
# First Block
# ======================
x = layers.Conv2D(128, (3, 3), activation='relu', padding="same")(inputs)
x = layers.Conv2D(128, (3, 3), activation='relu', padding="same")(x)
x = layers.MaxPooling2D((2, 2))(x)

# ======================
# Second Block
# ======================
x = layers.Conv2D(256, (3, 3), activation='relu', padding="same")(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding="same")(x)
x = layers.MaxPooling2D((2, 2))(x)

# ======================
# Parallel Branches
# ======================

# Branch 1
branch1 = layers.Conv2D(64, (3, 3), activation='relu', padding="same")(x)
branch1 = layers.Conv2D(64, (3, 3), activation='relu', padding="same")(branch1)

# Branch 2
branch2 = layers.Conv2D(64, (3, 3), activation='relu', padding="same")(x)
branch2 = layers.Conv2D(64, (3, 3), activation='relu', padding="same")(branch2)

# Concatenate feature maps
concat = layers.Concatenate()([branch1, branch2])

# ======================
# Fully Connected Layers
# ======================
flatten = layers.Flatten()(concat)
dense1 = layers.Dense(64, activation='relu')(flatten)

# --- NEW DROPOUT LAYER ADDED HERE ---
# Add a Dropout layer after the 64-neuron dense layer.
# A rate of 0.5 is a common starting point.
dropout = Dropout(0.5)(dense1)

# --- THIS IS THE CORRECTED LINE ---
# The final output layer now has 4 units to match your 4 classes
outputs = layers.Dense(4, activation='softmax')(dropout)

# Create the model
model = Model(inputs=inputs, outputs=outputs)
model.summary()

In [ ]:
print(train_generator.class_indices)

In [ ]:
# 6. Create and Compile the Model
model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 7. Print the Model Summary
print("Model Summary:")
model.summary()

In [ ]:
history = model.fit(
    train_generator,
    epochs= 30,
    validation_data=val_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_steps=val_generator.samples // BATCH_SIZE
)

In [ ]:
results = model.evaluate(test_generator,steps=len(test_generator), verbose=1)
print("Test Accuracy:", results[1])


In [ ]:
import numpy as np

# 2. Predictions
Y_pred = model.predict(test_generator, verbose=1)
y_pred = np.argmax(Y_pred, axis=1)

In [ ]:
# ======================
# 🔹 Save Weights
# ======================
model.save("/kaggle/working/brain_tumor.h5")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# 1. Get the true labels from the generator
true_labels = test_generator.classes

# 2. Make predictions (if you haven't already)
# This returns probabilities
probabilities = model.predict(test_generator)

# 3. Convert probabilities to predicted class labels
y_pred = np.argmax(probabilities, axis=1)

# 4. Create and plot the confusion matrix
cm = confusion_matrix(true_labels, y_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=test_generator.class_indices.keys(),
            yticklabels=test_generator.class_indices.keys())
plt.title("Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

In [ ]:
from sklearn.metrics import classification_report
print("\n Classification Report:\n")
print(classification_report(test_generator.classes, y_pred,
                            target_names=list(test_generator.class_indices.keys())))

In [ ]:
# 5. Per-class accuracy
cm_diag = np.diag(cm) / np.sum(cm, axis=1)
for idx, cls in enumerate(test_generator.class_indices.keys()):
    print(f"Class {cls}: {cm_diag[idx]*100:.2f}% accuracy")

In [ ]:
import matplotlib.pyplot as plt

# Get the data from the history object
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(accuracy))

# Plot Training and Validation Accuracy
plt.plot(epochs, accuracy, 'r', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'b', label='Validation accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.figure()

# Plot Training and Validation Loss
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and Validation Loss')
plt.legend()

plt.show()